In [1]:
import numpy as np
import pandas as pd
import gzip
import json

from pprint import pprint

In [2]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
turkish_stopwords = stopwords.words('turkish')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Osama\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
train_classification_df = pd.read_csv("/Users/Osama/Downloads/CS412PROJ/train-classification.csv",)
train_classification_df = train_classification_df.rename(columns={'Unnamed: 0': 'user_id', 'label': 'category'})

# Unifying labels
train_classification_df["category"] = train_classification_df["category"].apply(str.lower)
username2_category = train_classification_df.set_index("user_id").to_dict()["category"]

In [4]:
train_classification_df.groupby("category").count()

user_id
category                     
art                       191
entertainment             323
fashion                   299
food                      511
gaming                     13
health and lifestyle      503
mom and children          149
sports                    113
tech                      346
travel                    294

In [5]:
username2_category["kod8net"]

'tech'

In [6]:
'''# Load the additional CSV file
additional_data_path = "/Users/Osama/Downloads/CS412PROJ/annotated_users_CS412-2753ef4cf74e.csv"
additional_data_df = pd.read_csv(additional_data_path)

# Extract and rename the relevant columns
additional_data_df = additional_data_df[['url', 'influencerCategory']].rename(columns={
    'url': 'user_id', 
    'influencerCategory': 'category'
})

additional_data_df = additional_data_df.dropna()



# Convert the 'category' column to string type and apply .lower(), handling any NaN or unexpected values
additional_data_df['category'] = additional_data_df['category'].astype(str).fillna('').apply(str.lower)

train_classification_df = pd.read_csv("/Users/Osama/Downloads/CS412PROJ/train-classification.csv")

# Append the new data to the original train_classification_df
train_classification_df = pd.concat([train_classification_df, additional_data_df], ignore_index=True)

# Update the username2_category dictionary with the new data
username2_category.update(additional_data_df.set_index("user_id").to_dict()["category"])

# Check the updated data by viewing the first few rows
print(train_classification_df.head())

# Re-check the category distribution
print(train_classification_df.groupby("category").count())'''


'# Load the additional CSV file\nadditional_data_path = "/Users/Osama/Downloads/CS412PROJ/annotated_users_CS412-2753ef4cf74e.csv"\nadditional_data_df = pd.read_csv(additional_data_path)\n\n# Extract and rename the relevant columns\nadditional_data_df = additional_data_df[[\'url\', \'influencerCategory\']].rename(columns={\n    \'url\': \'user_id\', \n    \'influencerCategory\': \'category\'\n})\n\nadditional_data_df = additional_data_df.dropna()\n\n\n\n# Convert the \'category\' column to string type and apply .lower(), handling any NaN or unexpected values\nadditional_data_df[\'category\'] = additional_data_df[\'category\'].astype(str).fillna(\'\').apply(str.lower)\n\ntrain_classification_df = pd.read_csv("/Users/Osama/Downloads/CS412PROJ/train-classification.csv")\n\n# Append the new data to the original train_classification_df\ntrain_classification_df = pd.concat([train_classification_df, additional_data_df], ignore_index=True)\n\n# Update the username2_category dictionary with the 

In [7]:
train_data_path = "/Users/Osama/Downloads/CS412PROJ/training-dataset.jsonl.gz"

username2posts_train = dict()
username2profile_train = dict()

username2posts_test = dict()
username2profile_test = dict()


with gzip.open(train_data_path, "rt") as fh:
  for line in fh:
    sample = json.loads(line)

    profile = sample["profile"]
    username = profile["username"]
    if username in username2_category:
      # train data info
      username2posts_train[username] = sample["posts"]
      username2profile_train[username] = profile


    else:
      # it is test data info
      username2posts_test[username] = sample["posts"]
      username2profile_test[username] = profile

In [8]:
train_profile_df = pd.DataFrame(username2profile_train).T.reset_index(drop=True)
test_profile_df = pd.DataFrame(username2profile_test).T.reset_index(drop=True)

train_profile_df.head(2)

username          id    full_name  \
0  deparmedya  3170700063  Depar Medya   
1  kafesfirin   266439571  KAFES FIRIN   

                                           biography   category_name  \
0           #mediaplanning #mediabuying #sosyalmedya  Local business   
1  📍Söğütözü📍FTZ AVM\n🛒Ankara macro▲center v...           Brand   

  post_count follower_count following_count is_business_account is_private  \
0       None           1167             192                True      False   
1       None          11997              17                True      False   

   ... business_category_name overall_category_name category_enum  \
0  ...                   None                  None         LOCAL   
1  ...                   None                  None         BRAND   

  is_verified_by_mv4b is_regulated_c18  \
0               False            False   
1               False            False   

                                     profile_pic_url should_show_category  \
0  https://instagram.fsaw2-3.fna.fbcdn.net/v/t51....                 True   
1  https://instagram.fada1-13.fna.fbcdn.net/v/t51...                 True   

  should_show_public_contacts show_account_transparency_details  \
0                        True                              True   
1                        True                              True   

                              profile_picture_base64  
0  /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...  
1  /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...  

[2 rows x 44 columns]

In [9]:
test_profile_df.head(2)

username          id                    full_name  \
0              beyazyakaliyiz  8634457436           Selam Beyaz Yakalı   
1  totalenergies_istasyonlari  7066643793  TotalEnergies İstasyonları   

                                           biography   category_name  \
0        Beyaz yakalıların dünyasına hoşgeldiniz 😀😀😀   Personal blog   
1  TotalEnergies İstasyonları resmi Instagram hes...  Energy Company   

  post_count follower_count following_count is_business_account is_private  \
0       None           1265             665                True      False   
1       None          28025               4                True      False   

   ... business_category_name overall_category_name   category_enum  \
0  ...                   None                  None   PERSONAL_BLOG   
1  ...                   None                  None  ENERGY_COMPANY   

  is_verified_by_mv4b is_regulated_c18  \
0               False            False   
1               False            False   

                                     profile_pic_url should_show_category  \
0  https://instagram.fist6-1.fna.fbcdn.net/v/t51....                 True   
1  https://instagram.fsaw2-1.fna.fbcdn.net/v/t51....                 True   

  should_show_public_contacts show_account_transparency_details  \
0                        True                              True   
1                        True                              True   

                              profile_picture_base64  
0  /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...  
1  /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...  

[2 rows x 44 columns]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import emoji


def preprocess_text(text: str):
    

    # lower casing Turkish Text, Don't use str.lower :)
    text = text.casefold()

    #text = emoji.demojize(text, delimiters=(" ", " "))

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove special characters and punctuation
    # HERE THE EMOJIS stuff are being removed, you may want to keep them :D
    text = re.sub(r'[^a-zçğıöşü0-9\s#@]', '', text)

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text


corpus = []

# to keep the label order
train_usernames = []

for username, posts in username2posts_train.items():
  train_usernames.append(username)

  # aggregating the posts per user
  cleaned_captions = []
  for post in posts:
    post_caption = post.get("caption", "")
    if post_caption is None:
      continue

    post_caption = preprocess_text(post_caption)

    if post_caption != "":
      cleaned_captions.append(post_caption)


  # joining the posts of each user with a \n
  user_post_captions = "\n".join(cleaned_captions)
  corpus.append(user_post_captions)



#custom_stopwords = list(set(turkish_stopwords).union({
 #  'the', 'and', 'with', 'for', 'you', 'to', 'of', 'in', 'our', 'your', 'is', 'are','bir'
#}))
vectorizer = TfidfVectorizer(stop_words=turkish_stopwords, max_features=10000,min_df=10,ngram_range=(1, 3))

# fit the vectorizer
vectorizer.fit(corpus)

# transform the data into vectors
x_post_train = vectorizer.transform(corpus)
y_train = [username2_category.get(uname, "NA") for uname in train_usernames]
feature_names = vectorizer.get_feature_names_out()

# Inspect the frequency of each word
df_tfidf = pd.DataFrame(x_post_train.toarray(), columns=feature_names)

# Show the most frequent words (words in many posts)
print(df_tfidf.sum().sort_values(ascending=False).head(30))




test_usernames = []
test_corpus = []
for username, posts in username2posts_test.items():
  test_usernames.append(username)
  # aggregating the posts per user
  cleaned_captions = []
  for post in posts:
    post_caption = post.get("caption", "")
    if post_caption is None:
      continue

    post_caption = preprocess_text(post_caption)

    if post_caption != "":
      cleaned_captions.append(post_caption)

  user_post_captions = "\n".join(cleaned_captions)
  test_corpus.append(user_post_captions)


# Just transforming! No Fitting!!!!!
x_post_test = vectorizer.transform(test_corpus)

In [280]:
# Making sure everything is fine
assert y_train.count("NA") == 0

In [281]:
feature_names = vectorizer.get_feature_names_out()
feature_names

array(['ab', 'abd', 'abdulkadir', ..., 'şıklık', 'şıklığı', 'şımartın'],
      dtype=object)

In [282]:
df_tfidf = pd.DataFrame(x_post_train.toarray(), columns=feature_names)
df_tfidf.head(2)

ab  abd  abdulkadir  abdullah  abi  abiye  abone  about  about the  abs  \
0  0.0  0.0         0.0       0.0  0.0    0.0    0.0    0.0        0.0  0.0   
1  0.0  0.0         0.0       0.0  0.0    0.0    0.0    0.0        0.0  0.0   

   ...  şöyle  şükran  şükranla  şükranlarımızı  şükür       şık  şık bir  \
0  ...    0.0     0.0       0.0             0.0    0.0  0.041074      0.0   
1  ...    0.0     0.0       0.0             0.0    0.0  0.000000      0.0   

   şıklık  şıklığı  şımartın  
0     0.0      0.0       0.0  
1     0.0      0.0       0.0  

[2 rows x 10000 columns]

In [283]:
df_tfidf.shape

(2741, 10000)

In [284]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(df_tfidf, y_train, test_size=0.2, stratify=y_train)

In [285]:
x_train.shape

(2192, 10000)

In [286]:
x_val.shape

(549, 10000)

In [287]:
'''from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report


model = MultinomialNB()
model.fit(x_train, y_train)'''
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

import numpy as np
from collections import Counter

# Count the number of users in each class
class_counts = Counter(y_train)

# Total number of samples
total_samples = len(y_train)

# Calculate class weights using log10
'''log_class_weights = {
    cls: np.log10(1 + total_samples / count)  # Logarithmic scaling of class weight
    for cls, count in class_counts.items()
}
'''
unique_classes, class_counts = np.unique(y_train, return_counts=True)
log_class_weights = {cls: 1 / np.log10(1 + cnt) for cls, cnt in zip(unique_classes, class_counts)}
print("Class Weights with Log10 scaling:", log_class_weights)

class_weights = compute_class_weight(
    class_weight='balanced', 
    classes=np.unique(y_train), 
    y=y_train
)
class_weight_dict = {cls: weight for cls, weight in zip(np.unique(y_train), class_weights)}
print("Class Weights:", class_weight_dict)


from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix

# Initialize SVM model with class weights
svm_model = LinearSVC(class_weight=log_class_weights,max_iter=10,random_state=123)



#svm_model = LinearSVC(class_weight=class_weight_dict, max_iter=2000)
#non_linear_svm = SVC(kernel='rbf', class_weight=log_class_weights,max_iter=2000,random_state=42)
# Train the model
svm_model.fit(x_train, y_train)

Class Weights with Log10 scaling: {np.str_('art'): np.float64(0.4571385269519278), np.str_('entertainment'): np.float64(0.41437040473946346), np.str_('fashion'): np.float64(0.4201307776703473), np.str_('food'): np.float64(0.38273353435958046), np.str_('gaming'): np.float64(0.9602525677891274), np.str_('health and lifestyle'): np.float64(0.38383221246247784), np.str_('mom and children'): np.float64(0.48095855130519705), np.str_('sports'): np.float64(0.5104537371796872), np.str_('tech'): np.float64(0.4091578033553849), np.str_('travel'): np.float64(0.4214231285223929)}
Class Weights: {np.str_('art'): np.float64(1.4326797385620915), np.str_('entertainment'): np.float64(0.8496124031007752), np.str_('fashion'): np.float64(0.9171548117154812), np.str_('food'): np.float64(0.5359413202933986), np.str_('gaming'): np.float64(21.92), np.str_('health and lifestyle'): np.float64(0.545273631840796), np.str_('mom and children'): np.float64(1.842016806722689), np.str_('sports'): np.float64(2.435555555

C:\Users\Osama\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(class_weight={np.str_('art'): np.float64(0.4571385269519278),
                        np.str_('entertainment'): np.float64(0.41437040473946346),
                        np.str_('fashion'): np.float64(0.4201307776703473),
                        np.str_('food'): np.float64(0.38273353435958046),
                        np.str_('gaming'): np.float64(0.9602525677891274),
                        np.str_('health and lifestyle'): np.float64(0.38383221246247784),
                        np.str_('mom and children'): np.float64(0.48095855130519705),
                        np.str_('sports'): np.float64(0.5104537371796872),
                        np.str_('tech'): np.float64(0.4091578033553849),
                        np.str_('travel'): np.float64(0.4214231285223929)},
          max_iter=10, random_state=123)

In [288]:
'''from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Calculate class weights using log10
unique_classes, class_counts = np.unique(y_train, return_counts=True)
class_weights = {cls: 1 / np.log10(1 + cnt) for cls, cnt in zip(unique_classes, class_counts)}

print("Class Weights (log10 adjusted):", class_weights)

# Create Logistic Regression model
log_reg = LogisticRegression(
    multi_class='multinomial',  # for multiclass classification
    solver='lbfgs',            # efficient solver for multiclass problems
    class_weight=class_weights,  # handle class imbalance
    max_iter=1000               # allow sufficient iterations for convergence
)

# Train the model
log_reg.fit(x_train, y_train)

# Predict on validation set
y_val_pred = log_reg.predict(x_val)

# Evaluate the model
print("Classification Report:")
print(classification_report(y_val, y_val_pred))

# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_val, y_val_pred))
'''


'from sklearn.linear_model import LogisticRegression\nfrom sklearn.metrics import classification_report, confusion_matrix\nimport numpy as np\n\n# Calculate class weights using log10\nunique_classes, class_counts = np.unique(y_train, return_counts=True)\nclass_weights = {cls: 1 / np.log10(1 + cnt) for cls, cnt in zip(unique_classes, class_counts)}\n\nprint("Class Weights (log10 adjusted):", class_weights)\n\n# Create Logistic Regression model\nlog_reg = LogisticRegression(\n    multi_class=\'multinomial\',  # for multiclass classification\n    solver=\'lbfgs\',            # efficient solver for multiclass problems\n    class_weight=class_weights,  # handle class imbalance\n    max_iter=1000               # allow sufficient iterations for convergence\n)\n\n# Train the model\nlog_reg.fit(x_train, y_train)\n\n# Predict on validation set\ny_val_pred = log_reg.predict(x_val)\n\n# Evaluate the model\nprint("Classification Report:")\nprint(classification_report(y_val, y_val_pred))\n\n# Confus

In [289]:
'''from sklearn.model_selection import GridSearchCV
param_grid = {'svc__C': [1, 5, 10, 50],
              'svc__gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(svm_model, param_grid)

%time grid.fit(Xtrain, ytrain)
print(grid.best_params_)'''


"from sklearn.model_selection import GridSearchCV\nparam_grid = {'svc__C': [1, 5, 10, 50],\n              'svc__gamma': [0.0001, 0.0005, 0.001, 0.005]}\ngrid = GridSearchCV(svm_model, param_grid)\n\n%time grid.fit(Xtrain, ytrain)\nprint(grid.best_params_)"

In [290]:
#@title Train Data
y_train_pred = svm_model.predict(x_train)

print("Accuracy:", accuracy_score(y_train, y_train_pred))
print("\nClassification Report:")
print(classification_report(y_train, y_train_pred, zero_division=0))

Accuracy: 0.9913321167883211

Classification Report:
                      precision    recall  f1-score   support

                 art       1.00      0.97      0.99       153
       entertainment       1.00      0.99      0.99       258
             fashion       0.98      1.00      0.99       239
                food       0.99      1.00      0.99       409
              gaming       1.00      1.00      1.00        10
health and lifestyle       1.00      0.98      0.99       402
    mom and children       0.99      0.99      0.99       119
              sports       1.00      1.00      1.00        90
                tech       0.99      1.00      0.99       277
              travel       0.98      0.99      0.99       235

            accuracy                           0.99      2192
           macro avg       0.99      0.99      0.99      2192
        weighted avg       0.99      0.99      0.99      2192



In [291]:
y_val_pred = svm_model.predict(x_val)

print("Accuracy:", accuracy_score(y_val, y_val_pred))
print("\nClassification Report:")
print(classification_report(y_val, y_val_pred, zero_division=0))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_val_pred))

Accuracy: 0.6666666666666666

Classification Report:
                      precision    recall  f1-score   support

                 art       0.57      0.21      0.31        38
       entertainment       0.47      0.37      0.41        65
             fashion       0.74      0.72      0.73        60
                food       0.77      0.93      0.84       102
              gaming       0.00      0.00      0.00         3
health and lifestyle       0.62      0.70      0.66       100
    mom and children       0.62      0.70      0.66        30
              sports       0.74      0.74      0.74        23
                tech       0.56      0.67      0.61        69
              travel       0.81      0.71      0.76        59

            accuracy                           0.67       549
           macro avg       0.59      0.57      0.57       549
        weighted avg       0.66      0.67      0.65       549

Confusion Matrix:
 [[ 8  8  4  4  0  6  2  0  5  1]
 [ 1 24  4  8  0  9  2  

In [292]:
#@title Test Data
test_data_path = "/Users/Osama/Downloads/CS412PROJ/test-classification-round1.dat"

with open(test_data_path, "rt") as fh:
    for i, line in enumerate(fh):
        print(line.strip())
        if i >= 4:  # Stop after 5 lines
            break

print("*****")

test_unames = []
with open(test_data_path, "rt") as fh:
  for line in fh:
    test_unames.append(line.strip())

print(test_unames[:5])

ozhotelstr
elleturkiye
sozerinsaatorhangazi
sanliurfapiazzaavym
rusanozden
*****
['ozhotelstr', 'elleturkiye', 'sozerinsaatorhangazi', 'sanliurfapiazzaavym', 'rusanozden']


In [293]:
x_test = []

for uname in test_unames:
  try:
    index = test_usernames.index(uname)
    x_test.append(x_post_test[index].toarray()[0])
  except Exception as e:
    try:
      index = train_usernames.index(uname)
      x_test.append(x_post_train[index].toarray()[0])
    except Exception as e:
      print(uname)


test_unames.remove("screenname")

screenname


In [294]:
df_test = pd.DataFrame(np.array(x_test), columns=feature_names)
df_test.head(2)

ab       abd  abdulkadir  abdullah  abi  abiye  abone  about  about the  \
0  0.0  0.000000         0.0       0.0  0.0    0.0    0.0    0.0        0.0   
1  0.0  0.007874         0.0       0.0  0.0    0.0    0.0    0.0        0.0   

   abs  ...     şöyle  şükran  şükranla  şükranlarımızı  şükür       şık  \
0  0.0  ...  0.000000     0.0       0.0             0.0    0.0  0.000000   
1  0.0  ...  0.024696     0.0       0.0             0.0    0.0  0.010534   

   şık bir  şıklık  şıklığı  şımartın  
0      0.0     0.0      0.0       0.0  
1      0.0     0.0      0.0       0.0  

[2 rows x 10000 columns]

In [37]:
test_pred = svm_model.predict(df_test)

output = dict()
for index, uname in enumerate(test_unames):
  output[uname] = test_pred[index]

In [39]:
with open("output7.json", "w") as of:
  json.dump(output, of, indent=4)

In [247]:
import json

# Path to the test data file
test_data_path = "/Users/Osama/Downloads/CS412PROJ/test-regression-round1.jsonl"

# Read the test IDs
test_ids = []
with open(test_data_path, "r") as file:
    for line in file:
        data = json.loads(line.strip())
        test_ids.append(data["id"])  # Extract the "id" field from each line

print(test_ids[:5])

['18144550534306740', '17995331788956693', '18302703232191518', '17908564031282979', '17952070292635946']


In [297]:
output = {
    uname: test_pred[index].title() for index, uname in enumerate(test_unames)
}

# Write the output to a JSON file
output_file = "output3.json"
with open(output_file, "w") as of:
    json.dump(output, of, indent=4)

print(f"Predictions saved to {output_file}")

Predictions saved to output3.json
